## Downloading data from google cloud

In [ ]:
!wget https://storage.googleapis.com/kerascvnlp_data/young-affectnet-hq.zip

--2023-07-24 17:29:19--  https://storage.googleapis.com/kerascvnlp_data/young-affectnet-hq.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 172.253.117.128, 142.250.99.128, 142.250.107.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|172.253.117.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5441294496 (5.1G) [application/zip]
Saving to: ‘young-affectnet-hq.zip’

young-affectnet-hq. 100%[===================>]   5.07G   177MB/s    in 46s     

2023-07-24 17:30:06 (112 MB/s) - ‘young-affectnet-hq.zip’ saved [5441294496/5441294496]



In [ ]:
!unzip -q /content/young-affectnet-hq.zip -d data/

## installing and importing required libraries

In [ ]:
!pip install -q keras-core

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.1/753.1 kB 8.3 MB/s eta 0:00:00


## Building the [DAN model ](https://arxiv.org/pdf/2109.07270.pdf)

In [ ]:
import tensorflow as tf
from keras_core import Model, Sequential
from keras_core.layers import Layer
import keras_core.layers as nn
import keras_core as keras

class ChannelAttn(Layer):
    def __init__(self, c=512) -> None:
        super(ChannelAttn,self).__init__()
        self.gap = nn.AveragePooling2D(7)
        self.attention = Sequential([
            nn.Dense(32),
            nn.BatchNormalization(),
            nn.ReLU(),
            nn.Dense(c,activation='sigmoid')]
        )

    def call(self, x):

        x = self.gap(x)
        x = nn.Flatten()(x)
        y = self.attention(x)
        return x * y


class SpatialAttn(Layer):
    def __init__(self, c=512):
        super(SpatialAttn,self).__init__()
        self.conv1x1 = Sequential([
            nn.Conv2D(256, 1),
            nn.BatchNormalization()]
        )
        self.conv_3x3 = Sequential([
            nn.ZeroPadding2D(padding=(1, 1)),
            nn.Conv2D(512, 3,1),
            nn.BatchNormalization()]
        )
        self.conv_1x3 = Sequential([
            nn.ZeroPadding2D(padding=(0, 1)),
            nn.Conv2D(512, (1,3)),
            nn.BatchNormalization()]
        )
        self.conv_3x1 = Sequential([
            nn.ZeroPadding2D(padding=(1, 0)),
            nn.Conv2D(512,(3,1)),
            nn.BatchNormalization()]
        )
        self.norm = nn.ReLU()

    def call(self, x) :
        y = self.conv1x1(x)
        y = self.norm(self.conv_3x3(y) + self.conv_1x3(y) + self.conv_3x1(y))
        y = tf.math.reduce_sum(y,axis=1, keepdims=True)
        return x*y


class CrossAttnHead(Layer):
    def __init__(self, c=512):
        super(CrossAttnHead,self).__init__()
        self.sa = SpatialAttn(c)
        self.ca = ChannelAttn(c)

    def call(self, x):
        return self.ca(self.sa(x))


@keras.saving.register_keras_serializable(package='custom')
class DAN(Model):
    def __init__(self, num_classes=8,trainable=True,dtype='float32'):
        super(DAN,self).__init__()
        self.mod = keras.applications.ResNet50(
            include_top=False,
            weights="imagenet",
            input_shape=(224,224,3)
        )
        self.mod.trainable= False
        self.num_head = 4
        self.hd = CrossAttnHead()
        self.hd=[]
        for i in range(self.num_head):
          self.hd.append(CrossAttnHead())
        self.features = nn.Conv2D(512, 1,padding='same')
        self.fc = nn.Dense(num_classes)
        self.bn = nn.BatchNormalization()

    def call(self, x) :
        x = self.mod(x)
        x=self.features(x)
        heads = []
        for h in self.hd:
            heads.append(h(x))

        heads = tf.transpose(tf.stack(heads),perm=(1,0,2))
        heads = keras.ops.log_softmax(heads, axis=1)
        out = self.bn(self.fc(tf.math.reduce_sum(heads,axis=1)))
        return out

model = DAN()
img = tf.random.normal(shape=[10, 224, 224, 3])
preds = model(img)
print(preds.shape)

(10, 8)


creating the image dataloader using the keras-core utils function

In [ ]:
train_ds = keras.utils.image_dataset_from_directory(
            directory="data/",
            labels='inferred',
            label_mode='categorical',
            batch_size=32,
            image_size=(224, 224))

Found 14648 files belonging to 8 classes.


In [ ]:
model.compile(optimizer='adam',loss=keras.losses.CategoricalCrossentropy())
model.fit(train_ds,epochs=1)

## Saving and loading the model using inbuilt functions of keras_core

In [ ]:
model.save('weights.keras')

In [ ]:
pb = keras.saving.load_model('weights.keras')

/usr/local/lib/python3.10/dist-packages/keras_core/src/saving/saving_lib.py:338: UserWarning: Skipping variable loading for optimizer 'adam', because it has 190 variables whereas the saved optimizer has 2 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [ ]:
pred = pb(img)
pred.shape

TensorShape([10, 8])

In [ ]:
!zip -r wgts.zip weights.keras

In [ ]:
!wget https://storage.googleapis.com/kerascvnlp_data/danwgts/wgts.zip
!unzip wgts.zip

In [ ]:
!pip install tflite-support

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 390.3/390.3 kB 26.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Using cached pybind11-2.10.4-py3-none-any.whl (222 kB)
  Created wheel for tflite-support: filename=tflite_support-0.1.0a1-cp310-cp310-linux_x86_64.whl size=5942401 sha256=c7e9b35b7492f34e69eee330b1b65e327b281488484e256ff208e30908d0ddbb
  Stored in directory: /root/.cache/pip/wheels/71/5c/da/9e5e661ec26e03ee57e69428d40fffbefe3c0aff649c55776d
Successfully built tflite-support


In [ ]:
!python ./metadata_writer_for_image_classifier.py \
    --model_file=model.tflite \
    --label_file=labels.txt \
    --export_directory=/content/oiut

2023-05-26 16:26:33.200042: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Finished populating metadata and associated file to the model:
model.tflite
The metadata json file has been saved to:
/content/oiut/model.json
The associated file that has been been packed to the model is:
['labels.txt']


In [ ]:
import tensorflow as tf

# Convert the model
pb = keras.saving.load_model('weights.keras')
converter = tf.lite.TFLiteConverter.from_keras_model(pb) # path to the SavedModel directory
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

## References

[Distract Your Attention: Multi-head Cross Attention Network for Facial Expression Recognition](https://arxiv.org/pdf/2109.07270.pdf)

[Pytorch Implementation of DAN](https://github.com/yaoing/DAN)

[Official Keras Core Documentation](https://keras.io/keras_core/)